In [1]:
import json
import re
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from DeathtoGridSearch import GridSearch

In [2]:
%matplotlib inline

In [2]:
import pandas as pd

In [3]:
raw_data = [re.sub("[^a-zA-Z0-9.,]", "", i).split(',') for i in open("claim.sample.csv").read().strip().split("\n")]

In [4]:
columns = raw_data[0]

#map columns to the number which they are in
colmap = {k:v for v,k in enumerate(columns)}
colmap_r = {v:k for k,v in colmap.items()}

noheader = raw_data[1:]

## A medical claim is denoted by a claim number ('Claim.Number'). Each claim consists of one or more medical lines denoted by a claim line number ('Claim.Line.Number').

### 1. J-codes are procedure codes that start with the letter 'J'.

     A. Find the number of claim lines that have J-codes.

In [5]:
J_codes = 0
J_idxs = []

for rownum, row in enumerate(noheader):
#     print(row[colmap["Procedure.Code"]])
    try:
        if row[colmap['Procedure.Code']][0] == 'J':
            J_codes += 1
            J_idxs.append(rownum)
    except IndexError:
        continue

In [6]:
print("The number of J-Codes is: ", J_codes)

The number of J-Codes is:  51029


    B. How much was paid for J-codes to providers for 'in network' claims?

In [7]:
total_payment = 0

for rownum in J_idxs:
    total_payment += float(noheader[rownum][colmap["Provider.Payment.Amount"]])

In [8]:
total_payment

2418429.5728249843

     C. What are the top five J-codes based on the payment to providers?

In [9]:
codes_map = {}

for rownum in J_idxs:
    try:
        codes_map[noheader[rownum][colmap["Procedure.Code"]]] += \
                  float(noheader[rownum][colmap["Provider.Payment.Amount"]])
    except KeyError:
        codes_map[noheader[rownum][colmap["Procedure.Code"]]] = \
                  float(noheader[rownum][colmap["Provider.Payment.Amount"]])
            
codes_inv = {v:k for k,v in codes_map.items()}

In [10]:
print("The Top 5 J codes are... ")
for val in sorted(codes_map.values())[-5:]:
    print(codes_inv[val], " : ", val)

The Top 5 J codes are... 
J1644  :  81909.39601500003
J3490  :  90249.91244999997
J9310  :  168630.87357999996
J0180  :  299776.56076499994
J1745  :  434232.08058999997


### 2. For the following exercises, determine the number of providers that were paid for at least one J-code. Use the J-code claims for these providers to complete the following exercises.

    A. Create a scatter plot that displays the number of unpaid claims (lines where the ‘Provider.Payment.Amount’ field is equal to zero) for each provider versus the number of paid claims.    

In [11]:
jdf = [noheader[idx] for idx in J_idxs]

In [12]:
unpaid_claims = Counter()
paid_claims = Counter()

for row in jdf:
    if int(float(row[colmap["Provider.Payment.Amount"]])) == 0:
        unpaid_claims[row[colmap["Provider.ID"]]] += 1
    else:
        paid_claims[row[colmap["Provider.ID"]]] += 1

In [13]:
print(len(unpaid_claims.keys()))
print(len(paid_claims.keys()))

15
13


In [14]:
x = [paid_claims[k] for k in unpaid_claims.keys()]
y = [unpaid_claims[k] for k in unpaid_claims.keys()]

In [15]:
fig, ax = plt.subplots(1,1, figsize=(15,15))
ax.scatter(x, y);
ax.set_title("Paid Claims vs Unpaid Claims")
ax.set_xlabel("Paid Claims")
ax.set_ylabel("Unpaid Claims")
ax.set_ylim(-5,15000)
ax.set_xlim(-5,15000)

(-5, 15000)

    B. What insights can you suggest from the graph?

    C. Based on the graph, is the behavior of any of the providers concerning? Explain.

### 3. Consider all claim lines with a J-code.

    A. What percentage of J-code claim lines were unpaid?

In [16]:
unpaid_count = 0
paid_count = 0

for row in jdf:
    if int(float(row[colmap["Provider.Payment.Amount"]])) == 0:
        unpaid_count += 1
    else:
        paid_count += 1

In [17]:
print("{:.2f}% of all claims were unpaid".format(unpaid_count / (unpaid_count + paid_count) * 100))

88.30% of all claims were unpaid


    B. Create a model to predict when a J-code is unpaid. Explain why you choose the modeling approach.

In [18]:
model_cols = []

numeric_cols = [
    'Subscriber.Payment.Amount',
    'Claim.Charge.Amount',
]

dummy_cols = {
    "Provider.ID" : {},
    "Line.Of.Business.ID" : {},
    "Service.Code": {},
    "In.Out.Of.Network" : {},
    "Network.ID": {},
    "Agreement.ID" : {},
    "Price.Index": {},
    "Claim.Type": {},
    "Procedure.Code": {},
    "Revenue.Code": {}
}

for rownum, row in enumerate(jdf):
    for col in dummy_cols.keys():
        try:
            dummy_cols[col][rownum].add(row[colmap[col]])
        except KeyError:
            dummy_cols[col][row[colmap[col]]] = {rownum}

In [19]:
for colname, dumdict in dummy_cols.items():
    for dummy in dumdict.keys():
        model_cols.append("{}${}".format(colname, dummy))

In [20]:
model_df = []

for idx, row in enumerate(jdf):
    ith_row = []
    for colname in model_cols:
        col, val = colname.split("$")
        if idx in dummy_cols[col][val]:
            ith_row.append(1)
        else:
            ith_row.append(0)
    for numeric in numeric_cols:
        ith_row.append(float(row[colmap[numeric]]))
    if float(row[colmap["Provider.Payment.Amount"]]) > 0.00:
        ith_row.append(0.0)
    else:
        ith_row.append(1.0)
#     ith_row.append(float(row[colmap["Provider.Payment.Amount"]]) > 0)
    model_df.append(ith_row)

In [21]:
for numeric_col in numeric_cols:
    model_cols.append(numeric_col)

In [22]:
np_df = np.array(model_df)

In [23]:
true_vals = np_df[np_df[:,-1] == 1.0]
false_vals = np_df[np_df[:,-1] == 0.0]

In [24]:
np.random.shuffle(true_vals)

In [25]:
b_df = np.concatenate([false_vals, true_vals[:false_vals.shape[0]]])

In [26]:
np.random.shuffle(b_df)

In [27]:
b_df[:20]

array([[   0.     ,    0.     ,    0.     , ...,    0.     ,  135.0296 ,
           1.     ],
       [   0.     ,    0.     ,    0.     , ...,    0.     ,  493.8    ,
           1.     ],
       [   0.     ,    0.     ,    0.     , ...,    0.     ,  213.51912,
           1.     ],
       ..., 
       [   0.     ,    0.     ,    0.     , ...,    0.     ,  135.17775,
           1.     ],
       [   0.     ,    0.     ,    0.     , ...,    0.     ,  238.2585 ,
           1.     ],
       [   0.     ,    0.     ,    0.     , ...,    0.     ,  314.1185 ,
           0.     ]])

In [26]:
vals = np.random.random(b_df.shape[0])
folds = [vals < .2,
         ((vals >= .2) & (vals < .4)),
         ((vals >= .4) & (vals < .6)),
         ((vals >= .6) & (vals < .8)),
         vals >= .8]

In [27]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [28]:
print("LogisticRegression")
for fold in np.arange(len(folds)):
    model = LogisticRegression()
    model.fit(b_df[folds[fold], :-1], b_df[folds[fold], -1])
    predictions = model.predict(b_df[~folds[fold],:-1])
    print("\t", (predictions == b_df[~folds[fold],-1]).sum() / b_df[~folds[fold]].shape[0])

print("Random Forest Classifier")
for fold in np.arange(len(folds)):
    model = RandomForestClassifier(n_estimators=500, n_jobs=-1)
    model.fit(b_df[folds[fold], :-1], b_df[folds[fold], -1])
    predictions = model.predict(b_df[~folds[fold],:-1])
    print("\t", (predictions == b_df[~folds[fold],-1]).sum() / b_df[~folds[fold]].shape[0])
    
print("XG Boost Classifier")
for fold in np.arange(len(folds)):
    model = XGBClassifier(max_depth=5, n_estimators=500, n_jobs=-1)
    model.fit(b_df[folds[fold], :-1], b_df[folds[fold], -1])
    predictions = model.predict(b_df[~folds[fold],:-1])
    print("\t", (predictions == b_df[~folds[fold],-1]).sum() / b_df[~folds[fold]].shape[0])
    
print("Support Vector Classifier")
for fold in np.arange(len(folds)):
    model = SVC(degree=5)
    model.fit(b_df[folds[fold], :-1], b_df[folds[fold], -1])
    predictions = model.predict(b_df[~folds[fold],:-1])
    print("\t", (predictions == b_df[~folds[fold],-1]).sum() / b_df[~folds[fold]].shape[0])

LogisticRegression
	 0.512106788079
	 0.51343314192
	 0.512852276247
	 0.495628921115
	 0.515126569253
Random Forest Classifier
	 0.618687913907
	 0.625102543068
	 0.613296170125
	 0.623161575645
	 0.621012554023
XG Boost Classifier
	 0.603166390728
	 0.604901558655
	 0.598327655621
	 0.599814871953
	 0.601564107841
Support Vector Classifier
	 0.567570364238
	 0.561320754717
	 0.54588623929
	 0.557441118996
	 0.564210742951



    C. How accurate is your model at predicting unpaid claims?

    D. What data attributes are predominately influencing the rate of non-payment?

In [424]:
df.columns

Index(['Claim.Number', 'Claim.Line.Number', 'Member.ID', 'Provider.ID',
       'Line.Of.Business.ID', 'Revenue.Code', 'Service.Code',
       'Place.Of.Service.Code', 'Procedure.Code', 'Diagnosis.Code',
       'Claim.Charge.Amount', 'Denial.Reason.Code', 'Price.Index',
       'In.Out.Of.Network', 'Reference.Index', 'Pricing.Index',
       'Capitation.Index', 'Subscriber.Payment.Amount',
       'Provider.Payment.Amount', 'Group.Index', 'Subscriber.Index',
       'Subgroup.Index', 'Claim.Type', 'Claim.Subscriber.Type',
       'Claim.Pre.Prince.Index', 'Claim.Current.Status', 'Network.ID',
       'Agreement.ID'],
      dtype='object')

In [33]:
gs = GridSearch(b_df[:,:-1], b_df[:,-1])

In [53]:
gs.optimize_model('xgboost')

ValueError: cannot reshape array of size 127 into shape (10,10)

In [54]:
m = gs.models['xgboost']

In [58]:
m.tuning_options

{'max_depth': array([ 1,  4,  7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49]),
 'n_estimators': array([   1,    1,    1,    2,    3,    4,    5,    7,   10,   13,   17,
          23,   32,   42,   57,   76,  101,  135,  181,  241,  322,  430,
         574,  767, 1024])}

In [57]:
m.tuning_state

{'max_depth': 49, 'n_estimators': 1024}

In [ ]:
gs.save_results()

In [34]:
gs.optimize_all_models()
gs.save_results()

/Users/travis/Documents/smucode/DS7335/hwk1/DeathtoGridSearch.py:387: RuntimeWarning: invalid value encountered in true_divide
  recs.append( float(tp) / (tp + fn) )
/Users/travis/Documents/smucode/DS7335/hwk1/DeathtoGridSearch.py:387: RuntimeWarning: invalid value encountered in true_divide
  recs.append( float(tp) / (tp + fn) )
/Users/travis/Documents/smucode/DS7335/hwk1/DeathtoGridSearch.py:387: RuntimeWarning: invalid value encountered in true_divide
  recs.append( float(tp) / (tp + fn) )
/Users/travis/Documents/smucode/DS7335/hwk1/DeathtoGridSearch.py:387: RuntimeWarning: invalid value encountered in true_divide
  recs.append( float(tp) / (tp + fn) )
/Users/travis/Documents/smucode/DS7335/hwk1/DeathtoGridSearch.py:387: RuntimeWarning: invalid value encountered in true_divide
  recs.append( float(tp) / (tp + fn) )
/Users/travis/Documents/smucode/DS7335/hwk1/DeathtoGridSearch.py:387: RuntimeWarning: invalid value encountered in true_divide
  recs.append( float(tp) / (tp + fn) )
/Use

IndexError: index 17 is out of bounds for axis 0 with size 17

In [36]:
mod = gs.models['xgboost']

In [37]:
mod.numeric_state

[7, 17]

In [41]:
for i in mod.tuning_options.values():
    print(len(i))

25
17


In [43]:
gs.results_auc['xgboost'].shape[0]

25

In [47]:
gs.results_auc['xgboost'].size / 25

17.0

In [42]:
gs.results_auc['xgboost']

array([[ 0.51022554,  0.54872519,  0.        ,  0.60969983,  0.        ,
         0.67062594,  0.        ,  0.69266807,  0.69250266,  0.        ,
         0.69235156,  0.        ,  0.69235156,  0.        ,  0.69235156,
         0.        ,  0.69235156],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.64672067,
         0.67062594,  0.68527704,  0.69266807,  0.69250266,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.51022554,  0.54872519,  0.        ,  0.60969983,  0.64672067,
         0.67062594,  0.68527704,  0.69266807,  0.69250266,  0.        ,
         0.69235156,  0.        ,  0.69235156,  0.        ,  0.69235156,
         0.        ,  0.69235156],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.64782892,
         0.6748216 ,  0.68656942,  0.6909247 ,  0.69128735,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        

In [51]:
35 % 10

5